In [2]:
%matplotlib qt5
#%% Import modules
from mats_utils.rawdata.read_data import read_MATS_data
import datetime as DT
from mats_utils.plotting.plotCCD import *
from math import *
from mats_l1_processing.pointing import pix_deg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize_scalar
from skyfield.positionlib import ICRF
from skyfield.api import wgs84 
from skyfield.units import Distance
from skyfield import api as sfapi
from scipy.interpolate import RegularGridInterpolator

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [3]:
#%% Functions
def pix_deg2(ccditem, xpixel, ypixel):
    """
    Function to get the x and y angle from a pixel relative to the center of the CCD
    WARNING : no images are flipped in this function
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
    xpixel : int or array[int]
        x coordinate of the pixel(s) in the image
    ypixel : int or array[int]
        y coordinate of the pixel(s) in the image
        
    Returns
    -------
    xdeg : float or array[float]
        angular deviation along the x axis in degrees (relative to the center of the CCD)
    ydeg : float or array[float]
        angular deviation along the y axis in degrees (relative to the center of the CCD) 
    """
    h = 6.9 # height of the CCD in mm
    d = 27.6 # width of the CCD in mm
    # selecting effective focal length
    if (ccditem['CCDSEL']) == 7: # NADIR channel
        f = 50.6 # effective focal length in mm
    else: # LIMB channels
        f = 261    
    
    ncskip = ccditem['NCSKIP']
    try:
        ncbin = ccditem['NCBIN CCDColumns']
    except:
        ncbin = ccditem['NCBINCCDColumns']
    nrskip = ccditem['NRSKIP']
    nrbin = ccditem['NRBIN']
    ncol = ccditem['NCOL'] # number of columns in the image MINUS 1

    if (ccditem['CCDSEL']) in [1,3,5,6]:
        xpixel = ncol-xpixel
        
    yCCDpix = (nrskip + nrbin * (ypixel+0.5)) # y position of the pixel on the CCD (0.5 at the bottom, 510.5 on top)
    xCCDpix = (ncskip + ncbin * (xpixel+0.5)) # x position of the pixel on the CCD (0.5 on the left, 2047.5 on the right)
    
    xdeg = np.rad2deg(np.arctan(d*(xCCDpix/2048-0.5)/f)) # angular deviation along the x axis in degrees
    ydeg = np.rad2deg(np.arctan(h*(yCCDpix/511-0.5)/f)) # angular deviation along the y axis in degrees

    xdeg = np.rad2deg(np.arctan(d*(ncskip + ncbin * (xpixel+0.5) - 2047./2)/(2048*f)))
    return xdeg, ydeg


def deg_map(ccditem):
    """
    Function to get the x and y angular deviation map for each pixel of the image. 
    The deviation is given in degrees relative to the center of the CCD
    WARNING : no images are flipped before calculating the angular deviation
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
            
    Returns
    -------
    xmap : array[float]
        angular deviation map along the x axis in degrees (relative to the center of the CCD)
    ymap : array[float]
        angular deviation map along the y axis in degrees (relative to the center of the CCD) 
    """    
    im = ccditem['IMAGE']

    a,b = np.shape(im)
    X = range(b)
    Y = range(a)
    xpixel, ypixel = np.meshgrid(X,Y)
    xmap,ymap = pix_deg2(ccditem, xpixel, ypixel)
    return xmap,ymap

def deg_map_test(ccditem):
    """
    Function to get the x and y angular deviation map for each pixel of the image. 
    The deviation is given in degrees relative to the center of the CCD
    WARNING : no images are flipped before calculating the angular deviation
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
            
    Returns
    -------
    xmap : array[float]
        angular deviation map along the x axis in degrees (relative to the center of the CCD)
    ymap : array[float]
        angular deviation map along the y axis in degrees (relative to the center of the CCD) 
    """    
    im = ccditem['IMAGE']

    a,b = np.shape(im)
    X = range(b)
    Y = range(a)
    xpixel, ypixel = np.meshgrid(X,Y)
    xmap,ymap = pix_deg(ccditem, xpixel, ypixel)
    return xmap,ymap


def funheight_square(s, t, pos, FOV):
    """
    Function to get the distance between a point at position pos + s*FOV and the surface of the Geoid (wgs84 model),
     at time t.
    
    
    Parameters
    ----------
    s : float
        length along the straight line
    t : skyfield.timelib.Time
        time
    pos : array[float]
        position in space where the line starts (~position of MATS). Array of 3 position coordinates in m in the ICRF reference frame
    FOV : array[float]
        angle of the line (direction of the line), array of 3 elements in the IFRC reference frame
            
    Returns
    -------
    elevation**2 : float
        elevation squared of the point pos+s*FOV in m**2
    """
    newp = pos + s * FOV
    newp = ICRF(Distance(m=newp).au, t=t, center=399)
    return wgs84.subpoint(newp).elevation.m**2


def findsurface(t, pos, FOV):
    """
    Function to get the distance between a point at position pos and the surface of the Geoid (wgs84 model),
     at time t, along the line oriented along the FOV direction and starting at position pos
    
    
    Parameters
    ----------
    t : skyfield.timelib.Time
        time
    pos : array[float]
        position in space where the line starts (~position of MATS). Array of 3 position coordinates in m in the ICRF reference frame
    FOV : array[float]
        angle of the line (direction of the line), array of 3 elements in the IFRC reference frame
            
    Returns
    -------
    res : OptimizeResult object
        res.x is the distance found in m   
    """
    res = minimize_scalar(funheight_square, args=(t, pos, FOV), bracket=(3e5, 8e5))
    return res


def NADIR_geolocation(ccditem,x_step=2,y_step=2,interp_method='cubic'):
    """
    Function to get the latitude, longitude and solar zenith angle map for each pixel of the image.
    The values are calculated for some points and then interpolated for each pixel.
    WARNING : no images are flipped
    
    Parameters
    ----------
    ccditem : CCDitem
        measurement
    x_step : int
        step along the x-axis in the image between 2 sampled points used for interpolation. The default value is 2.
    y_step : int
        step along the y-axis in the image between 2 sampled points used for interpolation. The default value is 2.
    interp_method :
        interpolation method : 'linear', 'nearest', 'slinear', 'cubic', 'quintic' and 'pchip'
            
    Returns
    -------
    lat_map : array[float]
        map giving the latitude for each pixel in the image
    lon_map : array[float]
        map giving the longitude for each pixel in the image
    sza_map : array[float]
        map giving the solar zenith angle for each pixel in the image
    """
    im = ccditem['IMAGE']
    x_deg_map, y_deg_map = deg_map(ccditem) # creating angle deviation map for each pixel (degress)
    a,b = np.shape(im)

    metoOHB  = R.from_matrix([[0,0,-1],[0,-1,0],[-1,0,0]])
    ts=sfapi.load.timescale()
    t=ts.from_datetime(ccditem['EXPDate'].replace(tzinfo=sfapi.utc)) # exposure time  
    q=ccditem.afsAttitudeState
    quat=R.from_quat(np.roll(q,-1)) # quaternion of MATS attitude (for the LIMB imager) 
    pos=ccditem.afsGnssStateJ2000[0:3] # position of MATS
    
    xd = range(0,b,x_step) # sampled pixels on the x axis
    yd = range(0,a,y_step) # sampled pixels on the y axis
    LAT = np.zeros((len(yd),len(xd)))
    LON = np.zeros((len(yd),len(xd)))
    SZA = np.zeros((len(yd),len(xd)))

    # computing the latitude, longitude and solar zenith angles at the intersection of the line of sight and the earth surface
    # only the line of sights from some sampled pixels are computed
    for i in range(len(yd)):
        for j in range(len(xd)):
                x = xd[j]
                y = yd[i]
                # angular transformations
                # rotation from the line of sight of the LIMB imager to the line of sight of the NADIR pixel
                angle = R.from_euler('XYZ', [x_deg_map[y,x],-(90-23)+y_deg_map[y,x],0] , degrees=True).apply([1, 0, 0])
                FOV = quat.apply(metoOHB.apply(angle)) # attitude state for the line of sight of the NADIR pixel    
                # finding the distance between the point pos and the Geoid along the line of sight
                res = findsurface(t,pos,FOV)
                newp = pos + res.x * FOV 
                newp = ICRF(Distance(m=newp).au, t=t, center=399) # point at the intersection between the line of sight at the pixel and the Geoid surface
                LAT[i,j]=wgs84.subpoint(newp).latitude.degrees # latitude of the point
                LON[i,j]=wgs84.subpoint(newp).longitude.degrees # longitude of the point    

                # finding the solar zenith angle of the point
                planets = sfapi.load('de421.bsp')
                earth=planets['Earth']
                sun=planets['Sun']
                SZA[i,j]=90-((earth+wgs84.subpoint(newp)).at(t).observe(sun).apparent().altaz())[0].degrees
    
    # interpolating the results along all the pixels
    interp_lat = RegularGridInterpolator((yd,xd),LAT,method=interp_method,bounds_error=False,fill_value=None) 
    interp_lon = RegularGridInterpolator((yd,xd),LON,method=interp_method,bounds_error=False,fill_value=None)
    interp_sza = RegularGridInterpolator((yd,xd),SZA,method=interp_method,bounds_error=False,fill_value=None)

    X_map,Y_map = np.meshgrid(range(b),range(a))
    lat_map = interp_lat((Y_map,X_map))
    lon_map = interp_lon((Y_map,X_map))
    sza_map = interp_sza((Y_map,X_map))


    # plt.figure()
    # plt.title('Latitude')
    # plt.imshow(LAT)
    # plt.colorbar()

    # plt.figure()
    # plt.title('Longitude')
    # plt.imshow(LON)
    # plt.colorbar()

    # plt.figure()
    # plt.title('Solar Zenith Angle')
    # plt.imshow(SZA)
    # plt.colorbar()

    return(lat_map,lon_map,sza_map)

In [11]:
#%% local variables
# value of a saturated pixel 
sat_val = 32880

# times for start and stop
start_time = DT.datetime(2023, 1, 12, 3, 30, 0)
stop_time = DT.datetime(2023, 1, 12, 4, 0, 0)
start_time = DT.datetime(2023, 3, 13, 0, 0, 0)
stop_time = DT.datetime(2023, 3, 14, 0, 0, 0)

# filter selecting Nadir chanel
filter={'CCDSEL': [7,7]}

In [12]:
#%% reading mesurements
df1a= read_MATS_data(start_time, stop_time,filter,level='1a',version='0.5')
print(len(df1a))

# displaying keys
pd.set_option('display.max_rows', 100)
df1a.dtypes

5255


EXPDate                datetime64[ns, UTC]
OriginFile                          object
ProcessingTime         datetime64[ns, UTC]
RamsesTime             datetime64[ns, UTC]
QualityIndicator                     int32
LossFlag                             int32
VCFrameCounter                       int32
SPSequenceCount                      int32
TMHeaderNanoseconds                  int64
SID                                 object
RID                                 object
CCDSEL                               int32
EXPNanoseconds                       int64
WDWMode                             object
WDWInputDataWindow                  object
WDWOV                                int32
JPEGQ                                int32
FRAME                                int32
NROW                                 int32
NRBIN                                int32
NRSKIP                               int32
NCOL                                 int32
NCBINFPGAColumns                     int32
NCBINCCDCol

In [112]:
ccditem = df1a.iloc[0]
x,y = deg_map(ccditem)
xt,yt = deg_map_test(ccditem)
dx = x-xt
dy = y-yt
plt.figure()
plt.imshow(dx)
plt.figure()
plt.imshow(dy)

In [20]:
#%% geolocating images
from tqdm import tqdm

n = len(df1a)
a,b = np.shape(df1a.iloc[0]['IMAGE'])
lat_points = np.zeros((n,a,b))
lon_points = np.zeros((n,a,b))
sza_points = np.zeros((n,a,b))
im_points = np.zeros((n,a,b))


for i in tqdm(range(n)):
    ccditem = df1a.iloc[i]
    im = ccditem['IMAGE']
    lat_map,lon_map,sza_map = NADIR_geolocation(ccditem,x_step=8,y_step=4,interp_method='cubic')
    lat_points[i,:,:] = lat_map
    lon_points[i,:,:] = lon_map
    sza_points[i,:,:] = sza_map
    im_points[i,:,:] = im
    #print(f"image n# {i}/{n}")

  1%|          | 56/5255 [00:23<34:56,  2.48it/s]

In [18]:
data_crs = ccrs.PlateCarree()
plt.figure()
#projection = ccrs.Mollweide()
#projection = ccrs.Orthographic(central_longitude=0.0, central_latitude=0.0, globe=None)
projection = ccrs.Robinson()
ax = plt.axes(projection=projection)
ax.set_global()
ax.coastlines()
ax.gridlines()
for i in tqdm(range(0,5000,5)):
    ax.pcolor(lon_points[i,::2,::2],lat_points[i,:,::2],im_points[i,::2,::2], transform=data_crs)
#plt.pcolormesh(np.reshape(lon_points[0:100,:,:],(a,b*100)), np.reshape(lat_points[0:100,:,:],(a,b*100)), np.reshape(im_points[0:100,:,:],(a,b*100)))

plt.show()   

  0%|          | 0/1000 [00:00<?, ?it/s]


TypeError: Incompatible X, Y inputs to pcolor; see help(pcolor)

In [36]:
im1 = im_points[0,:,:]
lon1 = lon_points[0,:,:]
lat1 = lat_points[0,:,:]
print(lat1)

[[41.06773216 41.05404733 41.04046095 41.02697402 41.01358752 41.00030239
  40.98711954 40.97403989 40.96106429 40.9481936  40.93542861 40.92277011
  40.91021885 40.89777554 40.88544087 40.87321549 40.8611     40.84909499
  40.83720099 40.8254185  40.81374799 40.80218987 40.79074453 40.7794123
  40.7681935  40.75708836 40.74609713 40.73521995 40.72445697 40.71380827
  40.7032739  40.69285385 40.68254808 40.67235652 40.66227901 40.65231541
  40.64246548 40.63272897 40.62310558 40.61359497 40.60419675 40.59491049
  40.58573574 40.57667199 40.56771869 40.55887525 40.55014107 40.54151547
  40.53299777 40.52458723 40.5162831  40.50808457 40.49999082 40.49200099
  40.48411418 40.47632946]
 [41.11766049 41.10390553 41.09025239 41.07670212 41.0632557  41.04991413
  41.03667835 41.0235493  41.01052787 40.99761494 40.98481134 40.97211789
  40.95953536 40.9470645  40.934706   40.92246055 40.91032878 40.89831129
  40.88640865 40.87462136 40.86294991 40.85139474 40.83995625 40.82863481
  40.8174307

In [9]:
import rasterio
from rasterio.transform import Affine

lon1

lonres = (lon1[0,-1] - lon1[0,0]) / lon1.shape[1]
latres = (lat1[-1,0] - lat1[0,0]) / lat1.shape[0]

transform = Affine.translation(lon1[0,0] - lonres / 2, lat1[0,0] - latres / 2) * Affine.scale(lonres, latres)

print(transform)

with rasterio.open(
        "/home/louis/MATS/MATS-analysis/Louis/rasterio/test.tif",
        mode="w",
        driver="GTiff",
        height=im1.shape[0],
        width=im1.shape[1],
        count=1,
        dtype=im1.dtype,
        crs="+proj=latlong",
        transform=transform,
) as new_dataset:
        new_dataset.write(im1, 1)


| 0.11, 0.00, 23.18|
| 0.00,-0.05, 66.07|
| 0.00, 0.00, 1.00|


In [10]:
from scipy.interpolate import griddata

def simple_resampler(data,lat,lon,new_lat,new_lon):
    a,b = np.shape(lon)
    points = np.zeros((a*b,2))
    points[:,0] = lon.ravel()
    points[:,1] = lat.ravel()
    new_data = griddata(points, data.ravel(), (new_lon, new_lat), method='nearest')
    return new_data


points = np.zeros((14*56,2))
points[:,0] = lon1.ravel()
points[:,1] = lat1.ravel()

lo = np.linspace(np.min(lon1),np.max(lon1),56*10)
la = np.linspace(np.min(lat1),np.max(lat1),14*10)
new_lon,new_lat = np.meshgrid(lo,la)
new_im = simple_resampler(im1,lat1,lon1,new_lat,new_lon)



plt.figure('original')
ax = plt.axes(projection=ccrs.PlateCarree())
plt.pcolor(lon1,lat1,im1)
ax.coastlines()
plt.show() 

plt.figure('resampled')
ax = plt.axes(projection=ccrs.PlateCarree())
plt.pcolor(new_lon,new_lat,new_im)
ax.coastlines()
plt.show() 


In [11]:
from scipy.interpolate import griddata

def simple_resampler_cube(data,lat,lon,new_lat,new_lon):
    a,b,c = np.shape(lon)
    points = np.zeros((a*b*c,2))
    points[:,0] = lon.ravel()
    points[:,1] = lat.ravel()
    new_data = griddata(points, data.ravel(), (new_lon, new_lat), method='cubic')
    return new_data


lo = np.linspace(44.89,55.9,100)
la = np.linspace(14.06,21,53,100)
new_lon,new_lat = np.meshgrid(lo,la)
new_im = simple_resampler_cube(im_points,lat_points,lon_points,new_lat,new_lon)



plt.figure('original')
ax = plt.axes(projection=ccrs.PlateCarree())
for i in tqdm(range(n)):
    #plt.pcolor(lon_points[i,:,:],lat_points[i,:,:],im_points[i,:,:])
    plt.scatter(lon_points[i,:,:].ravel(),lat_points[i,:,:].ravel(),im_points[i,:,:].ravel())
#plt.pcolormesh(np.reshape(lon_points[0:100,:,:],(a,b*100)), np.reshape(lat_points[0:100,:,:],(a,b*100)), np.reshape(im_points[0:100,:,:],(a,b*100)))
ax.coastlines()
plt.show() 

plt.figure('resampled')
ax = plt.axes(projection=ccrs.PlateCarree())
plt.pcolor(new_lon,new_lat,new_im)
ax.coastlines()
plt.show() 

100%|██████████| 10/10 [00:00<00:00, 454.75it/s]


In [12]:
import rasterio as rio
from rasterio.transform import from_gcps
from rasterio.control import GroundControlPoint as GCP

ul = (10.0000000, 30.0000000)  # in lon, lat / x, y order
ll = (80.7106781, -40.7106781)
ur = (80.7110000, 100.711)
lr = (151.4213562, 30.0000000)
cols, rows = 20, 20

gcps = [
    GCP(0, 0, lon1[0,0], lat1[0,0]),
    GCP(0, 56, lon1[0,55], lat1[0,55]),
    GCP(14, 0, lon1[13,0], lat1[13,0]),
    GCP(14, 56, lon1[13,55], lat1[13,55,])
]

transform = from_gcps(gcps)

print(transform)

with rasterio.open(
        "/home/louis/MATS/MATS-analysis/Louis/rasterio/test.tif",
        mode="w",
        driver="GTiff",
        height=im1.shape[0],
        width=im1.shape[1],
        count=1,
        dtype=im1.dtype,
        crs="+proj=latlong",
        transform=transform,
) as new_dataset:
        new_dataset.write(im1, 1)





| 0.10,-0.04, 23.34|
|-0.02,-0.05, 66.03|
| 0.00, 0.00, 1.00|


In [14]:
raster = rasterio.open("/home/louis/MATS/MATS-analysis/Louis/rasterio/test.tif")
raster_im = raster.read(1)

X,Y = np.meshgrid(range(56),range(14))
raster_lon,raster_lat = rasterio.transform.xy(raster.transform,Y,X)


plt.figure('original')
ax = plt.axes(projection=ccrs.PlateCarree())
plt.pcolor(lon1,lat1,im1)
ax.coastlines()
plt.show() 

plt.figure('rasterio')
ax = plt.axes(projection=ccrs.PlateCarree())
plt.pcolor(raster_lon,raster_lat,raster_im)
ax.coastlines()
plt.show() 



In [87]:
degmap = np.ones((4,4))
degmap[3,2] = 3
degmap

angles = np.zeros((16,3))
angles = [degmap.ravel()[:],0,0]
angles


[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1.]), 0, 0]